In [1]:
import mlflow

# Step 2: Set up the MLflow tracking server
# mlflow.set_tracking_uri("http://ec2-3-81-236-116.compute-1.amazonaws.com:5000/")

# Step 2: Set up the MLflow tracking server - local
mlflow.set_tracking_uri("http://127.0.0.1:5000/")

# Set or create an experiment
mlflow.set_experiment("Exp 5 - ML Algos with HP Tuning")

import optuna
import mlflow
import mlflow.sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import SMOTE, ADASYN
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

df = pd.read_csv("dataset.csv").dropna(subset=["clean_comment"])
df.shape
#  Step 1: Remap the class labels from [-1, 0, 1] to [2, 0, 1]
df["category"] = df["category"].map({-1: 2, 0: 0, 1: 1})

# Step 2: Remove rows where the target labels (category) are NaN
df = df.dropna(subset=["category"])

ngram_range = (1, 3)  # Trigram setting
max_features = 1000  # Set max_features to 1000 for TF-IDF

# Step 4: Train-test split before vectorization and resampling
X_train, X_test, y_train, y_test = train_test_split(
    df["clean_comment"], df["category"], test_size=0.2, random_state=42, stratify=df["category"]
)

# Step 2: Vectorization using TF-IDF, fit on training data only
vectorizer = TfidfVectorizer(ngram_range=ngram_range, max_features=max_features)
X_train_vec = vectorizer.fit_transform(X_train)  # Fit on training data
X_test_vec = vectorizer.transform(X_test)        # Transform test data

adasyn = ADASYN(random_state=42)
X_train_resampled, y_train_resampled = adasyn.fit_resample(X_train_vec, y_train)

c:\Data-Science\CampusX\Projects\youtube-comment-analyzer\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Function to log results in MLflow
def log_mlflow(model_name, model, X_train, X_test, y_train, y_test):
    with mlflow.start_run():
        # Log model type
        mlflow.set_tag("mlflow.runName", f"{model_name}_ADASYSN_TFIDF_Trigrams")
        mlflow.set_tag("experiment_type", "algorithm_comparison")

        # Log algorithm name as a parameter
        mlflow.log_param("algo_name", model_name)

        # Train model
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

        # Log accuracy
        accuracy = accuracy_score(y_test, y_pred)
        mlflow.log_metric("accuracy", accuracy)

        # Log classification report
        classification_rep = classification_report(y_test, y_pred, output_dict=True)
        for label, metrics in classification_rep.items():
            if isinstance(metrics, dict):
                for metric, value in metrics.items():
                    mlflow.log_metric(f"{label}_{metric}", value)

        # Log the model
        mlflow.sklearn.log_model(model, f"{model_name}_model")


# Step 6: Optuna objective function for SVM
def objective_svm(trial):
    C = trial.suggest_float("C", 1e-4, 10.0, log=True)
    kernel = trial.suggest_categorical("kernel", ["linear", "rbf", "poly"])

    model = SVC(C=C, kernel=kernel, random_state=42)
    return accuracy_score(
        y_test, model.fit(X_train_resampled, y_train_resampled).predict(X_test_vec)
    )


# Step 7: Run Optuna for SVM, log the best model only
def run_optuna_experiment():
    study = optuna.create_study(direction="maximize")
    study.optimize(objective_svm, n_trials=30)

    # Get the best parameters and log only the best model
    best_params = study.best_params
    best_model = SVC(C=best_params["C"], kernel=best_params["kernel"], random_state=42)

    # Log the best model with MLflow, passing the algo_name as "SVM"
    log_mlflow("SVM", best_model, X_train_resampled, X_test_vec, y_train_resampled, y_test)


# Run the experiment for SVM
run_optuna_experiment()

[I 2024-12-02 11:35:09,188] A new study created in memory with name: no-name-6a9e6254-d4ab-440f-9ee4-171a1afa836c
[I 2024-12-02 11:40:36,439] Trial 0 finished with value: 0.42629210418655394 and parameters: {'C': 0.005240968653455074, 'kernel': 'rbf'}. Best is trial 0 with value: 0.42629210418655394.
[I 2024-12-02 11:46:09,264] Trial 1 finished with value: 0.786035728896768 and parameters: {'C': 7.535969124870114, 'kernel': 'linear'}. Best is trial 1 with value: 0.786035728896768.
[I 2024-12-02 11:48:50,969] Trial 2 finished with value: 0.7674894313377881 and parameters: {'C': 0.14928017511260505, 'kernel': 'linear'}. Best is trial 1 with value: 0.786035728896768.
[I 2024-12-02 11:51:00,412] Trial 3 finished with value: 0.49679530887767626 and parameters: {'C': 0.02102231488485986, 'kernel': 'poly'}. Best is trial 1 with value: 0.786035728896768.
[I 2024-12-02 11:53:12,979] Trial 4 finished with value: 0.4301104595663439 and parameters: {'C': 0.00017141181997523528, 'kernel': 'linear'}

🏃 View run SVM_ADASYSN_TFIDF_Trigrams at: http://127.0.0.1:5000/#/experiments/602507174562575349/runs/8a4b1b7fe6434020834d17b449874307
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/602507174562575349
